In [2]:
import numpy as np
import pandas as pd

# sklearn tools
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [3]:
data = pd.read_csv('/Users/vaibhavrangan/Downloads/Stat_303-3/Datasets/heart_disease_classification.csv')

data.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
# Response and predictors
y = data['target']
X = data.drop('target', axis=1)

In [5]:
# Separate the data into training and test sets

# Inputs:
    # data: X, y
    # Ratio of the datasets: using 80%-20% below
    # random_state for reproducibility
    # stratify: to keep the Class 0 to Class 1 ratio the same on the training and test sets
        # This input is very important for classifiers, not necessary for regressors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=20, stratify = y)

In [6]:
print(y_train.value_counts()/y_train.shape[0])
print(y_test.value_counts()/y_test.shape[0])

target
1    0.545455
0    0.454545
Name: count, dtype: float64
target
1    0.540984
0    0.459016
Name: count, dtype: float64


In [7]:
# IF NECESSARY -- scale the data
scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Cross Validation Functions

cross_val_score

In [8]:
# An array of hyperparam values to try
Ks = np.arange(10, 160, 10)

# An empty list to store the CV scores
scores = []

for K in Ks: # For each hyperparam value
    model = KNeighborsClassifier(n_neighbors=K) # Create the model -- keeping weights as default
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5) # Using the model and data, perform 5-fold cv
    scores.append(cv_scores) # Store the results

# The output of the function is a score for each fold -- accuracy for a classifier by default, can be changed

In [9]:
avg_cv_scores = np.array(scores).mean(axis=1)

print('Best avg CV score:', max(avg_cv_scores)*100)
print('Best K:', Ks[avg_cv_scores.argmax()])

Best avg CV score: 82.25340136054422
Best K: 30


cross_val_predict

In [17]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import accuracy_score

# first let's do the greedy method, using the K value found above

tuned_model = KNeighborsClassifier(n_neighbors=Ks[avg_cv_scores.argmax()]) # hyperparameter already tuned

# cross_val_predict to get the cv prediction probabilities, so we can tune the threshold with them
# cross_val_predict output is the training data predictions WHEN EACH OBSERVATION WAS IN THE ASSESSMENT FOLD during cv
cv_probs = cross_val_predict(tuned_model, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]

# method='predict_proba' makes the function return probabilities, not class predictions

# tuning the threshold
thrs = np.arange(0, 1.01, 0.01)

accs = []

for thr in thrs: # iterate through the thresholds
    cv_preds = cv_probs > thr # get the cv preds for each threshold
    acc = accuracy_score(y_train, cv_preds) # get the cv accuracy
    accs.append(acc) # store the results

print(np.array(accs).max()) # best accuracy
print(thrs[np.array(accs).argmax()]) # best threshold

0.8223140495867769
0.5


Not guaranteed to be the best combination because we haven't tuned threshold with different K hyperparameters

In [18]:
# the guaranteed method

# K and threshold arrays
Ks = np.arange(10, 160, 10)
thrs = np.arange(0, 1.01, 0.01)

cv_results = pd.DataFrame(columns=['K', 'threshold', 'accuracy']) # empty dataframe to store cv results

counter = 0

for K in Ks: # iterate through the K values
    model = KNeighborsClassifier(n_neighbors=K) # create the model for specific K value
    cv_probs = cross_val_predict(model, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]
    
    for thr in thrs: # iterate through the thresholds
        cv_results.loc[counter, "K"] = K
        cv_results.loc[counter, "threshold"] = thr
        cv_results.loc[counter, "accuracy"] = accuracy_score(y_train, cv_probs > thr)
        counter += 1

cv_results.head()



,K,threshold,accuracy
0,10,0.0,0.636364
1,10,0.01,0.636364
2,10,0.02,0.636364
3,10,0.03,0.636364
4,10,0.04,0.636364


In [22]:
best_index = cv_results["accuracy"].argmax() # row index of best K and threshold combo

cv_results.iloc[best_index]

# greedy method missed the optimal combination

K                  20
threshold        0.55
accuracy     0.826446
Name: 156, dtype: object

In [23]:
len(cv_results)

1515

cross_validate

In [26]:
# if we want to find the cv results for multiple metrics at the same time, we can use cross_validate

from sklearn.model_selection import cross_validate

Ks = np.arange(10, 160, 10)

scores = []

for K in Ks:
    model = KNeighborsClassifier(n_neighbors=K)
    score = cross_validate(model, X_train_scaled, y_train, cv=5, scoring=['accuracy', 'precision', 'recall'])
    scores.append(score)

df_scores = pd.DataFrame(scores) # the test_... columns are of interest -- they are CV scores not test scores
df_scores

,fit_time,score_time,test_accuracy,test_precision,test_recall
0,"[0.0013871192932128906, 0.0006308555603027344,...","[0.0051000118255615234, 0.00774693489074707, 0...","[0.8775510204081632, 0.8163265306122449, 0.812...","[0.92, 0.8214285714285714, 0.7931034482758621,...","[0.8518518518518519, 0.8518518518518519, 0.884..."
1,"[0.0004248619079589844, 0.0003800392150878906,...","[0.0030422210693359375, 0.0029430389404296875,...","[0.8571428571428571, 0.7346938775510204, 0.854...","[0.8846153846153846, 0.7058823529411765, 0.806...","[0.8518518518518519, 0.8888888888888888, 0.961..."
2,"[0.00034499168395996094, 0.0003309249877929687...","[0.00267791748046875, 0.0027709007263183594, 0...","[0.8571428571428571, 0.7346938775510204, 0.875...","[0.8571428571428571, 0.7058823529411765, 0.833...","[0.8888888888888888, 0.8888888888888888, 0.961..."
3,"[0.0003209114074707031, 0.00030684471130371094...","[0.0027060508728027344, 0.0024759769439697266,...","[0.8775510204081632, 0.7551020408163265, 0.812...","[0.8888888888888888, 0.7142857142857143, 0.774...","[0.8888888888888888, 0.9259259259259259, 0.923..."
4,"[0.0003139972686767578, 0.0002949237823486328,...","[0.002508878707885742, 0.0024929046630859375, ...","[0.8571428571428571, 0.7142857142857143, 0.833...","[0.8571428571428571, 0.6756756756756757, 0.8, ...","[0.8888888888888888, 0.9259259259259259, 0.923..."
5,"[0.00030517578125, 0.00028967857360839844, 0.0...","[0.0025148391723632812, 0.002499103546142578, ...","[0.8367346938775511, 0.7142857142857143, 0.833...","[0.8518518518518519, 0.6756756756756757, 0.8, ...","[0.8518518518518519, 0.9259259259259259, 0.923..."
6,"[0.0003898143768310547, 0.0002808570861816406,...","[0.002554178237915039, 0.002468109130859375, 0...","[0.8979591836734694, 0.7142857142857143, 0.833...","[0.8666666666666667, 0.6756756756756757, 0.8, ...","[0.9629629629629629, 0.9259259259259259, 0.923..."
7,"[0.00034999847412109375, 0.0002923011779785156...","[0.002683877944946289, 0.0025420188903808594, ...","[0.9183673469387755, 0.6938775510204082, 0.833...","[0.896551724137931, 0.6578947368421053, 0.8, 0...","[0.9629629629629629, 0.9259259259259259, 0.923..."
8,"[0.00028395652770996094, 0.0002832412719726562...","[0.002621173858642578, 0.0026350021362304688, ...","[0.8979591836734694, 0.6938775510204082, 0.854...","[0.8666666666666667, 0.6578947368421053, 0.806...","[0.9629629629629629, 0.9259259259259259, 0.961..."
9,"[0.00024008750915527344, 0.0002818107604980469...","[0.0025360584259033203, 0.002838134765625, 0.0...","[0.8571428571428571, 0.6938775510204082, 0.854...","[0.8125, 0.6578947368421053, 0.806451612903225...","[0.9629629629629629, 0.9259259259259259, 0.961..."


In [27]:
df_scores["avg_cv_accuracy"] = df_scores["test_accuracy"].apply(np.mean)
df_scores["avg_cv_precision"] = df_scores["test_precision"].apply(np.mean)
df_scores["avg_cv_recall"] = df_scores["test_recall"].apply(np.mean)
df_scores

,fit_time,score_time,test_accuracy,test_precision,test_recall,avg_cv_accuracy,avg_cv_precision,avg_cv_recall
0,"[0.0013871192932128906, 0.0006308555603027344,...","[0.0051000118255615234, 0.00774693489074707, 0...","[0.8775510204081632, 0.8163265306122449, 0.812...","[0.92, 0.8214285714285714, 0.7931034482758621,...","[0.8518518518518519, 0.8518518518518519, 0.884...",0.822109,0.819090,0.871510
1,"[0.0004248619079589844, 0.0003800392150878906,...","[0.0030422210693359375, 0.0029430389404296875,...","[0.8571428571428571, 0.7346938775510204, 0.854...","[0.8846153846153846, 0.7058823529411765, 0.806...","[0.8518518518518519, 0.8888888888888888, 0.961...",0.822534,0.795223,0.917379
2,"[0.00034499168395996094, 0.0003309249877929687...","[0.00267791748046875, 0.0027709007263183594, 0...","[0.8571428571428571, 0.7346938775510204, 0.875...","[0.8571428571428571, 0.7058823529411765, 0.833...","[0.8888888888888888, 0.8888888888888888, 0.961...",0.822534,0.790562,0.924786
3,"[0.0003209114074707031, 0.00030684471130371094...","[0.0027060508728027344, 0.0024759769439697266,...","[0.8775510204081632, 0.7551020408163265, 0.812...","[0.8888888888888888, 0.7142857142857143, 0.774...","[0.8888888888888888, 0.9259259259259259, 0.923...",0.818197,0.783049,0.932194
4,"[0.0003139972686767578, 0.0002949237823486328,...","[0.002508878707885742, 0.0024929046630859375, ...","[0.8571428571428571, 0.7142857142857143, 0.833...","[0.8571428571428571, 0.6756756756756757, 0.8, ...","[0.8888888888888888, 0.9259259259259259, 0.923...",0.805952,0.772527,0.924501
5,"[0.00030517578125, 0.00028967857360839844, 0.0...","[0.0025148391723632812, 0.002499103546142578, ...","[0.8367346938775511, 0.7142857142857143, 0.833...","[0.8518518518518519, 0.6756756756756757, 0.8, ...","[0.8518518518518519, 0.9259259259259259, 0.923...",0.797704,0.770143,0.909402
6,"[0.0003898143768310547, 0.0002808570861816406,...","[0.002554178237915039, 0.002468109130859375, 0...","[0.8979591836734694, 0.7142857142857143, 0.833...","[0.8666666666666667, 0.6756756756756757, 0.8, ...","[0.9629629629629629, 0.9259259259259259, 0.923...",0.809949,0.770373,0.939316
7,"[0.00034999847412109375, 0.0002923011779785156...","[0.002683877944946289, 0.0025420188903808594, ...","[0.9183673469387755, 0.6938775510204082, 0.833...","[0.896551724137931, 0.6578947368421053, 0.8, 0...","[0.9629629629629629, 0.9259259259259259, 0.923...",0.805782,0.767848,0.939316
8,"[0.00028395652770996094, 0.0002832412719726562...","[0.002621173858642578, 0.0026350021362304688, ...","[0.8979591836734694, 0.6938775510204082, 0.854...","[0.8666666666666667, 0.6578947368421053, 0.806...","[0.9629629629629629, 0.9259259259259259, 0.961...",0.805867,0.763161,0.947009
9,"[0.00024008750915527344, 0.0002818107604980469...","[0.0025360584259033203, 0.002838134765625, 0.0...","[0.8571428571428571, 0.6938775510204082, 0.854...","[0.8125, 0.6578947368421053, 0.806451612903225...","[0.9629629629629629, 0.9259259259259259, 0.961...",0.797704,0.752328,0.947009


In [28]:
df_scores.index = Ks
df_scores.head()

,fit_time,score_time,test_accuracy,test_precision,test_recall,avg_cv_accuracy,avg_cv_precision,avg_cv_recall
10,"[0.0013871192932128906, 0.0006308555603027344,...","[0.0051000118255615234, 0.00774693489074707, 0...","[0.8775510204081632, 0.8163265306122449, 0.812...","[0.92, 0.8214285714285714, 0.7931034482758621,...","[0.8518518518518519, 0.8518518518518519, 0.884...",0.822109,0.819090,0.871510
20,"[0.0004248619079589844, 0.0003800392150878906,...","[0.0030422210693359375, 0.0029430389404296875,...","[0.8571428571428571, 0.7346938775510204, 0.854...","[0.8846153846153846, 0.7058823529411765, 0.806...","[0.8518518518518519, 0.8888888888888888, 0.961...",0.822534,0.795223,0.917379
30,"[0.00034499168395996094, 0.0003309249877929687...","[0.00267791748046875, 0.0027709007263183594, 0...","[0.8571428571428571, 0.7346938775510204, 0.875...","[0.8571428571428571, 0.7058823529411765, 0.833...","[0.8888888888888888, 0.8888888888888888, 0.961...",0.822534,0.790562,0.924786
40,"[0.0003209114074707031, 0.00030684471130371094...","[0.0027060508728027344, 0.0024759769439697266,...","[0.8775510204081632, 0.7551020408163265, 0.812...","[0.8888888888888888, 0.7142857142857143, 0.774...","[0.8888888888888888, 0.9259259259259259, 0.923...",0.818197,0.783049,0.932194
50,"[0.0003139972686767578, 0.0002949237823486328,...","[0.002508878707885742, 0.0024929046630859375, ...","[0.8571428571428571, 0.7142857142857143, 0.833...","[0.8571428571428571, 0.6756756756756757, 0.8, ...","[0.8888888888888888, 0.9259259259259259, 0.923...",0.805952,0.772527,0.924501


In [32]:
# return at least 94% cv recall with highest possible cv precision
df_filtered = df_scores[["avg_cv_accuracy", "avg_cv_precision", "avg_cv_recall"]]

# filter for avg_cv_recall above 94%
df_filtered2 = df_filtered[df_filtered["avg_cv_recall"] > 0.94]

# filter for highest avg_cv_precision
df_filtered2.loc[df_filtered2["avg_cv_precision"].idxmax()]

avg_cv_accuracy     0.805867
avg_cv_precision    0.763161
avg_cv_recall       0.947009
Name: 90, dtype: float64

- what if we want to tune with multiple metrics like this, but tuning the threshold as well?
- then we cannot use cross_validate

cv objects

GridSearchCV

In [33]:
from sklearn.model_selection import GridSearchCV

model = KNeighborsClassifier()

grid = {"n_neighbors": np.arange(10, 160, 10), "weights": ["uniform", "distance"]}

gscv = GridSearchCV(model, grid, cv=5, scoring="accuracy")

gscv.fit(X_train_scaled, y_train)

# .fit cross validates the model for each combo in the grid (15 K values * 2 weights = 30 combo)
# best values are found, the tuned model with these values is trained and ready to predict

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150]),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [35]:
print(gscv.best_params_)

# best avg CV score
print(gscv.best_score_)

{'n_neighbors': np.int64(30), 'weights': 'uniform'}
0.8225340136054422


after finding the tuned model:
    - for regression tuning is done
    - for classification, we need to tune the threshold as well
    - take gscv.best_estimator_ and use cross_val_predict as above

In [37]:
cv_probs = cross_val_predict(gscv.best_estimator_, X_train_scaled, y_train, cv=5, method='predict_proba')[:, 1]
thresholds = np.arange(0, 1.01, 0.01)

accs = []

for thr in thresholds:
    cv_preds = cv_probs > thr
    acc = accuracy_score(y_train, cv_preds)
    accs.append(acc)

print(np.array(accs).max())
print(thresholds[np.array(accs).argmax()])

0.8223140495867769
0.5
